In [2]:
import requests
import json
import concurrent.futures
import time
from tqdm import tqdm
import pandas as pd

In [3]:
categories_req = requests.get('https://letsreadasia.org/api/tag/total-books-and-tags')
categories_json= categories_req.json()

In [4]:
list_id = [item['id'] for item in categories_json['tags']]
[item['name'] for item in categories_json['tags']]

['Family & Friendship',
 'Superhero',
 'Problem Solving',
 'Community',
 'Critical Thinking',
 'Adventure',
 'Animals',
 'Non-fiction',
 'Nature',
 'Funny',
 'Mighty Girls',
 'Health',
 'Arts and Music',
 'Science',
 'Folktales']

In [5]:
list_book_urls = [f"https://letsreadasia.org/api/tag/get-books/{id}?limit=10000&lId=6260074016145408&cursor="
                  for id in list_id]
list_book_urls

['https://letsreadasia.org/api/tag/get-books/5760672340115456?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5765598403362816?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5728125824532480?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5715808797851648?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5765734416252928?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5699190361423872?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5728283396145152?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5697992333983744?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5726225838374912?limit=10000&lId=6260074016145408&cursor=',
 'https://letsreadasia.org/api/tag/get-books/5762464356

In [6]:
requests.get(list_book_urls[0]).json()

{'books': [{'id': '3b9f7254-e319-4c51-9141-82db0b02cd7b',
   'name': 'Bukan Halangan',
   'description': 'Kapisa mendapat haid pertamanya. Meskipun dia sudah tahu apa yang harus dilakukan, tetap saja perasaannya campur-aduk. Belum lagi pertandingan basket sudah di depan mata.',
   'thumborCoverImageUrl': 'https://letsread-images.hamropatro.com/75N5UnRMSVAldAb0x-vPxzitPn0=/300x400/smart/http://storage.googleapis.com/lets-read-asia/assets/images/76a696e7-69ba-40e7-8884-fb6072c81c21.jpg',
   'coverImageUrl': 'http://storage.googleapis.com/lets-read-asia/assets/images/76a696e7-69ba-40e7-8884-fb6072c81c21.jpg',
   'coverImageServingUrl': 'https://lh3.googleusercontent.com/2Glig-YXcE8GLTBTXP1DNnICG7ItFaiYiSbDKLQ371p3skv8Py1wKtouFMyv-s6s3k3wK98TZkE9q1oNkUbQBfuFhEaokVVqAcwL',
   'languageId': '6260074016145408',
   'language': {'createdDate': 1484313531246,
    'lastUpdated': 1664464104790,
    'id': 6260074016145408,
    'name': 'Bahasa Indonesia',
    'country': 'Indonesia',
    'languageISO

In [7]:
books_data = []

def get_data(url):
    resp = requests.get(url=url)
    if resp.status_code == 200:
        json_data = resp.json()
        return json_data['books']
    else:
        print(resp.text)
    

tm1 = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:

    futures = []

    for url in list_book_urls:
        futures.append(executor.submit(get_data, url=url))

    for future in tqdm(concurrent.futures.as_completed(futures)):
        books_data.extend(future.result())

tm2 = time.perf_counter()
len(books_data)

15it [00:12,  1.17it/s]


1141

In [8]:
df = pd.DataFrame(books_data)
df = df.drop_duplicates(subset=['id'])
df = df.drop(columns=['language','pdfUrl','tags'])
df

,id,name,description,thumborCoverImageUrl,coverImageUrl,coverImageServingUrl,languageId,signLanguageId,signLanguage,masterBookId,...,hasAudio,hasVideo,epubUrl,pages,originalUrl,translationAvailable,translationStatus,countryOfOrigin,favorite,featured
0,bdd514f4-008b-4a1a-95bd-ef2ee32e7e86,Ketupat Imbiuku,Ada imbiuku istimewa yang diinginkan Sinan dal...,https://letsread-images.hamropatro.com/O6-3xUg...,http://storage.googleapis.com/lets-read-asia/a...,https://lh3.googleusercontent.com/xcOboKe8Dtj3...,6260074016145408,None,None,bdd514f4-008b-4a1a-95bd-ef2ee32e7e86,...,False,False,https://storage.googleapis.com/lets-read-asia/...,None,None,True,None,Indonesia,False,False
1,e1335064-5236-4073-a9a2-8e1b53a364b5,Kelinci Cokelat yang Cerdik,"Seorang anak kecil merajuk untuk digendong, da...",https://letsread-images.hamropatro.com/Vz0P8nD...,http://storage.googleapis.com/lets-read-asia/a...,https://lh3.googleusercontent.com/7nYBvktzmQ7q...,6260074016145408,None,None,bc70b8c3-b2ef-452b-b0f3-a01286f4e564,...,False,False,https://storage.googleapis.com/lets-read-asia/...,None,None,True,None,Viet Nam,False,False
2,ed518c38-377d-40dd-8556-81357fafabc7,Manusia Super Penyelamat Kucing,Bo akan menjadi Manusia Super dan menyelamatka...,https://letsread-images.hamropatro.com/3YE1qbe...,http://storage.googleapis.com/lets-read-asia/a...,https://lh3.googleusercontent.com/66fQCH48cqVG...,6260074016145408,None,None,d07eabf2-58d0-420c-bffa-d5b3f4ce5f6c,...,False,False,https://storage.googleapis.com/lets-read-asia/...,None,None,True,None,Viet Nam,False,False
3,d547040a-8821-47ca-807d-e003a054cd7d,"Superhero Kita, Edhi Baba",Cerita bergambar ini menyajikan langkah-langka...,https://letsread-images.hamropatro.com/q2CnWqg...,http://storage.googleapis.com/lets-read-asia/a...,https://lh3.googleusercontent.com/wy3UhpF-JNva...,6260074016145408,None,None,6b899dec-6691-4225-aa5a-7454d7e6aa3d,...,False,False,https://storage.googleapis.com/lets-read-asia/...,None,None,True,None,,False,False
4,bf74951a-e358-4ecf-97d9-a92fab57b621,Roti Harapan,"Goretti sahabat Keona tidak sekolah lagi, kare...",https://letsread-images.hamropatro.com/iuagpGi...,http://storage.googleapis.com/lets-read-asia/a...,https://lh3.googleusercontent.com/ZrmkYcxvpEub...,6260074016145408,None,None,bf74951a-e358-4ecf-97d9-a92fab57b621,...,False,False,https://storage.googleapis.com/lets-read-asia/...,None,None,True,None,Indonesia,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,be2e57b8-3237-4ff9-bd9e-4e60c95eca60,Hujan,Apa kamu merasa nyaman di rumah saat hujan tur...,https://letsread-images.hamropatro.com/C_hiKaC...,http://storage.googleapis.com/lets-read-asia/a...,https://lh3.googleusercontent.com/t6bSsTnim-v6...,6260074016145408,None,None,be2e57b8-3237-4ff9-bd9e-4e60c95eca60,...,False,False,https://storage.googleapis.com/lets-read-asia/...,None,None,True,None,Indonesia,False,False
1136,b51eee3c-e283-4a8c-9a1b-e0033d41c8b6,Cerita Pohon,Ini kisah tentang biji mungil yang tumbuh menj...,https://letsread-images.hamropatro.com/JiJ7wSe...,http://storage.googleapis.com/lets-read-asia/a...,https://lh3.googleusercontent.com/5Km0ov_zoC7o...,6260074016145408,None,None,8eca608c-0ace-4f4e-9980-a574f6e7719e,...,False,False,https://storage.googleapis.com/lets-read-asia/...,None,None,True,None,,False,False
1138,69219a4b-bfdc-4016-b004-b471831fde1f,Mangga Besar Bersari,Sebuah puisi tentang mangga mulai dari awal hi...,https://letsread-images.hamropatro.com/OTEszRS...,http://storage.googleapis.com/lets-read-asia/a...,https://lh3.googleusercontent.com/bL0flwyWbysx...,6260074016145408,None,None,6188fde6-c565-4ac8-83cb-60a469617503,...,False,False,https://storage.googleapis.com/lets-read-asia/...,None,None,True,None,Nepal,False,False
1139,7477af93-a585-4afc-bdbb-2b039686b919,Bunty dan Pasukan Gelembung,"Bunty tidak suka sabun, maka dia pun menolak m...",https://letsread-images.hamropatro.com/qM6b2B-...,http://storage.googleapis.com/lets-read-asia/a...,https:

In [9]:
df.to_csv('letsreadasia_book.csv', index=False)

In [10]:
with open('letsreadasia_book.json', 'w') as f:
    json.dump(books_data, f)

### Pages

In [12]:
books_with_pages_json = []

complete_book_urls = [f"https://letsreadasia.org/api/v5/book/preview/language/6260074016145408/book/{id}" for id in df['masterBookId']]

def get_books_data(url):
    resp = requests.get(url=url)
    if resp.status_code == 200:
        json_data = resp.json()
        return json_data
    else:
        print(resp.text)
    

tm1 = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:

    futures = []

    for url in complete_book_urls:
        futures.append(executor.submit(get_books_data, url=url))

    for future in tqdm(concurrent.futures.as_completed(futures)):
        books_with_pages_json.append(future.result())

tm2 = time.perf_counter()
len(books_with_pages_json)
with open('letsreadasia_complete_books.json', 'w') as f:
    json.dump(books_with_pages_json, f)

498it [00:40, 12.35it/s]
